In [1]:
from json import loads 
from datetime import datetime
from uuid import UUID
from gzip import GzipFile
from io import TextIOWrapper
from json import dumps
from csv import reader
from pathlib import Path
from pyspark import SparkConf, SparkContext
from urllib.parse import quote, parse_qsl, SplitResult, urlsplit
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor

In [2]:
conf = SparkConf()
conf.setAll([
    ("spark.executor.instances", 3)
])
sc = SparkContext(
    master="yarn",
    appName="web-archive-query-log-stats",
    conf=conf,
)
sc

<SparkContext master=yarn appName=web-archive-query-log-stats>

In [3]:
alexa_services = """
google
youtube
baidu
qq
facebook
tmall
taobao
yahoo
amazon
wikipedia
jd
360
weibo
reddit
netflix
instagram
vk
microsoft
csdn
bing
twitter
twitch
zoom
ebay
naver
aliexpress
yandex
linkedin
bongacams
apple
pornhub
mail
stackoverflow
msn
tribunnews
imdb
livejasmin
chaturbate
dropbox
ok
xvideos
github
cnn
etsy
xhamster
sogou
canva
tumblr
espn
instructure
indeed
roblox
imgur
flipkart
fandom
bbc
detik
booking
cnblogs
walmart
alibaba
freepik
zhihu
nih
force
salesforce
stackexchange
daum
udemy
onlinesbi
savefrom
indiatimes
theguardian
craigslist
avito
grid
duckduckgo
aliyun
tiktok
primevideo
shutterstock
speedtest
mediafire
xnxx
gome
w3schools
researchgate
cnet
tokopedia
youm7
globo
slideshare
tistory
varzesh3
wikihow
quora
office365
bukalapak
ask
intuit
usps
redd
steamcommunity
deepl
airbnb
bankofamerica
wikimedia
blackboard
rambler
steampowered
investopedia
9gag
chegg
kakao
glassdoor
naukri
sourceforge
webmd
youdao
dbs
seznam
chinaz
ecosia
rediff
goo
turkiye
europa
dcinside
news18
gov
elfagr
bandcamp
allrecipes
123rf
fivethirtyeight
stanford
commbank
smartsheet
caf
4shared
lifewire
24h
baskino
tdameritrade
sfgate
gob
etrade
vectorstock
csod
fmovies
biggo
sagepub
pochta
tasnimnews
cyberleninka
hangseng
lg
semanticscholar
listindiario
nsw
startpage
zdf
postermywall
jagranjosh
berkeley
gotporn
dailypost
psu
akhbarelyom
prensalibre
indosport
iz
worldstarhiphop
guru99
suumo
calculator
virgilio
sapo
idealo
thebalancecareers
dpreview
tim
tnaflix
las2orillas
excite
9anime
windy
wetter
techtudo
books
kaiserpermanente
le360
euronews
itau
wiocha
utoronto
gitlab
commentcamarche
biglobe
jf71qh5v14
glosbe
eluniversal
akurat
monster
sportzwiki
arabi21
xm
rei
ci123
adyen
b2b168
thoughtco
iefimerida
wa
republicworld
mostaghelonline
elyamnelaraby
beauty321
popbela
finn
acfun
arzdigital
bd-pratidin
doc88
masrawy
php
taroot-rangi
amarujala
yixiin
xataka
julian-fashion
onliner
licindia
to10
nbg
acs
filmibeat
usembassy
nifty
skynewsarabia
agenziaentrate
knowyourmeme
filgoal
donanimhaber
ashemaletube
ixxx
programiz
careers360
myfonts
monografias
worldbank
zulily
klix
ew
warcraftlogs
yellowpages
dy2018
vlive
answers
ilfattoquotidiano
thumbzilla
arynews
wetteronline
sportbox
19888
info
igg-games
xmtrading
ubuntu
nyc
nikkansports
navyfederal
filmweb
docker
doramatv
watanserb
movieweb
leagueofgraphs
channelmyanmar
besoccer
razer
royanews
ulta
meteofrance
khabarfarsi
pc6
haibunda
opensubtitles
ampproject
citibank
leo
thomann
alarab
corporatefinanceinstitute
01net
all-free-download
mydailymagazine
sworld
alanba
pornhd
naijaloaded
wenxuecity
storyblocks
sbnation
milenio
fajar
walla
win-rar
islcollective
thebalance
usmagazine
weheartit
fitbit
domestika
careerbuilder
financialexpress
colorado
clubic
searchmulty
20minutes
thespruce
npmjs
arstechnica
whois
ufreegames
bseindia
proquest
balenciaga
boyfriendtv
deutsche-bank
slidesgo
puma
nudevista
uspto
sketchfab
vodafone
uptvs
pornhat
societegenerale
furaffinity
netacad
nipic
check24
fashionnova
playground
virgool
marieclaire
joemonster
qwant
thairath
powerschool
pearsonvue
denverpost
uefa
celine
tcs
cima4u
sbi
w3
shippingchina
secnews
tsite
interieur
gib
aglasem
zalando
jbhifi
chefkoch
microcenter
yourdailysportfix
sec
dim
tv9telugu
serverfault
3isk
sciencemag
dicio
bunshun
sedo
fzg360
simplilearn
tovima
redwap
manualslib
groww
fosshub
tineye
ixl
investors
javbus
ilna
loc
bradesco
thefappeningblog
msdmanuals
fuq
sarvgyan
anyporn
simplyhired
xhamsterpremium
diretta
coolmathgames
podbean
newgrounds
tebyan
paheal
sportzbonanza
dailypakistan
edf
bigbasket
iwank
coolors
instant-gaming
ovh
jiayuan
kongfz
babyshop
18183
3bmeteo
e621
interactivebrokers
bottegaveneta
arvancloud
movistarplus
club-k
bina
upmedia
travelermaster
elecfans
tailorbrands
russianfood
life
tuasaude
iconfinder
themoviedb
jiomart
xvideos2
dhl
duosecurity
3isk
pcworld
jiwu
afternic
notebookcheck
altibbi
planetsuzy
zaful
po-kaki-to
ig
focus
latam
anysex
geihui
patagonia
verycd
steamdb
animixplay
bonanza
mql5
keezmovies
9to5mac
usa
etudiant
al-maraabimedias
justice
alison
darty
theringer
tueren-fachhandel
nesn
zimbio
niksalehi
ehow
pornmd
2ch-c
mtv
setareparsi
tweaktown
softfamous
hsbianma
zaobao
givemesport
internshala
jogos360
belgium
todayhumor
dice
tubegalore
nationwide
sunporno
serebii
irrawaddy
khabarvarzeshi
arca
soccer24
javhdporn
letpub
areavip
eztv
opendns
brave
thenextweb
dreamhost
ck365
thegospelcoalition
iq
wgu
levi
hochi
yalla-shoot-new
radiotimes
dev
brstej
thisav
olevod
kinsta
aftodioikisi
stardewvalleywiki
thomasnet
dafiti
letyshops
linkvertise
999
ac-versailles
exey
manganato
hrblock
kizi
jobartis
ap
photokade
hdrezka
actblue
letmejerk
pardot
cjdropshipping
chittorgarh
gidonline
xitongcheng
subf2m
plesk
tigerdirect
fatosdesconhecidos
dmhy
ants
moniban
letour
9anime
empflix
asurascans
luxuretv
hi
kuai8
nsfc
ftc
websiteseochecker
shiprocket
powerapp
al-marsd
peardeck
wikimapia
discovercard
aldar
5i8xkqjmqubv
searchdimension
gimy
0123movie
influencermarketinghub
jra
2checkout
imna
bauhaus
jamaran
senate
hunter
educacao
webcrawler
kooora365
moviesjoy
17zwd
prweb
almashhad-alyemeni
fortinet
dygang
yts-subs
activecampaign
loveplanet
upworthy
diagrams
povar
shayanews
santander
pingone
ntv
ssl2anyone3
chime
investorplace
freeones
limestart
agemys
gogoanime
legifrance
rozee
gaymaletube
sciencealert
vavel
bestjavporn
vijesti
everydaykoala
codelist
joq
wowprogress
yoo7
elmwatin
decolar
yummly
yt5s
hongkiat
divxtotal3
mybookie
mangaraw
gazetadopovo
tax
6park
perezhilton
smashingmagazine
ero-video
y2meta
iflscience
porntube
spyfu
launchpage
aps
gogoanime
dygod
americanmilitarynews
btctrademart
alot
voachinese
eanswers
gogoanime
buedemusica
archlinux
insideevs
tvsubtitles
pornhdvideos
seoclerk
gogoanime
bkrs
openai
natro
rvcj
ccn
18comic
moretify
pornez
yyets
cashstar
yupptv
fgov
searchengineland
pronto
ucloud
rolloid
4movierulz
playjunkie
goodporn
wkzf
movies2watch
auchan
pianshen
weakstreams
futebolplayhd
yingjiesheng
kat
bank-zone
ic
shahed4u
rule34video
gaoding
watchfreejavonline
thevintagenews
orient
folkd
salesloft
kbhgames
redditmedia
schema
etheplatinum
kmart
infocert
fanpop
sexbjcam
shahed4u
hearthstonetopdecks
nios
mostkbal
searchlock
news19
tagdiv
livingsocial
bestmining
znzmo
unza
newsd
futemax
sexkbj
angola24horas
51pla
mcls
jacquielawson
higherperspectives
news-sphere
beautifultrendstoday
asianembed
notjustok
pptvhd36
bestdealfor28
sidereel
sopitas
searchalgo
healthdata
limetorrents
imovie-hd
nqrkzcd7ixwr
pishnahadevizheh
largeporntube
mojnews
workercn
weaplay
hentaihaven
webhostingtalk
4movierulz
dontorrent
zk
her-news
programmersought
khoondanionline
torrent9
diguver
pirateproxy
aufeminin
hotpoptoday
gbfile
64p3am9x95ct
quicklisti
alriyadh
resumersvo
lassuranceretraite
in
jp
javdb39
pornerbros
1plus1
pornmedium
search-ch
bjeea
dygang
agadirinfo
hibasport
dogpile
amateur8
pron
justporno
mybeautyland
d72gb9oukw3j
bmony
ligueimovel
123movies
maturetubehere
porntop
khabarnama
mxss
freesion
movierulzfree
naijanews
ws1xr1u2b4
elfagr
sexseq
gamingwonderland
jomys
maktoob
chinaso
comicat
faithtap
mmbang
vid
mangafox
hellporno
falafelandcaviar
sarcasm
oreno-erohon
7movierulz
putlockers
freefilefillableforms
metbugat
moe
slashgear
dilidili
ibtimes
jmcomic
bestblackfriday
javdb36
download-adblock
windows
fanli
covid-19-testingkit
ces
espnfc
3movierulz
westernjournalism
aswaqinformation
fanpage
elitetorrent
trovi
hostgator
stagram
sfwaa
mmamania
q6idnawboy7g
asg
blackberry
bloodyelbow
plugrush
seobook
byrutor
justindianporn
musicasparamissa
youtu
recommendationletters
123greetings
betterex
ibtimes
kwork
er
unblocked
nflstreams
apherald
teamliquid
torrent9
browserquote
jobstreet
angovagas
covidvisualizer
searchengines
cooch
canaltutorial
ilividnewtab
gostream
hentai-time
jmcomic
pudding
dyndns
wn
shodan
woothemes
filesonic
unblocked
tennessean
opensiteexplorer
unblocked
xxxhdvideo
ctitv
turkmenistanhabar
liftable
imzog
thoughclassifiedjeff
4udear
dunia21
probux
inis
majesticseo
guj5
viralnova
metacrawler
7movierulz
mycricketlive
hdrezka
ukcontentdelivery
tickld
injuredcandy
podregarddomicile
websta
xgap
anahwa
pretagteam
mbank
sergey-mavrodi
nhadatso
skat
yeus
cr447
forosdelweb
seb
kiddle
pussl18
ecrater
localgirldating
eurovision
sportsala
egotastic
ah-me
exceptingincludedelivering
onlyfreetoonporn
fillsitsy
uploading
sxc
ellechina
masslistener
tamilrockers
6movierulz
lycos
anightsregalia
ebi
mayoclinic
grella
technorati
understandingharmoniousillegal
shopping
dontorrent
manga9
lauriegrowingdrops
hyves
citeab
porn24
admtpmp127
searchengines
default-search
reptilefightearn
pornhost
tuberel
alsbbora
ryzex
nyaa
govome
forexmetatradexdx
yhdmw
mobfactory
100ppi
filmstreaming2
ct10000
pleasedbeginparish
wapwon
twindiversity
avaxhome
royal
cryptoplace
snickgainfulmuch
how01
basketball24
jsoftj
tv-links
slideplayer
leninetudo
zooplus
autoproyecto
markosweb
parsijoo
m5zn
oomall
bdpcc
1movierulz
kelkoo
molodgytot
7movierulz
bhj1
shentai
hotsale
misrjournal
shopzilla
fmovies
pricegrabber
blinkx
tu
globalspec
petalsearch
lollipop-network
4movierulz
suite101
cima4u
3file
watchseries
cp
molebeagleheadless
surveyflsh
badjojo
astercom
mocah
statmyweb
1001jogos
icij
textile
123movies
earthday
adzuna
foxcdn
deferscoop
mediaoffers
dontorrent
5movierulz
7movierulz
grumbleoh
1xbet-164837
mybeautylands
pricerunner
torrentwiz1
you
dunia21
trafficunit
business
neosportek8
andcareyo
onemanga
vietxx
thegudda
coulderlyy
velsoftwa
letassigne
lo
eadingenered
sufficulumcu
giggsyunited
heroturko
kino
sourcecodester
anentryle
football3
xxxcomics
messyster
5movierulz
gobookee
btcmanager
oureviewi
jetboobs
wazap
trendymalldeals
maandhave
eyourcom
taskthesa
lenglishiam
fishcod
mychicconsulting
sandmyre
ficanportio
soverthlaest
picsearch
distemshu
tronglyre
kidzsearch
najdi
metager
omgili
gigablast
mojeek
evi
args
askmenow
base-search
books.google
chatnoir
chemrefer
egerin
findsounds
fireball
genieknows
leit
miner
munax
music.amazon
mysimon
ncbi
news.google
news.yahoo
newsandmoods
newslookup
nextag
pixsta
podscope
qmamu
recipebridge
scholar.google
seeqpod
songza
swisscows
veveo
wordpress.org/openverse
""".strip().splitlines()
len(alexa_services)

1086

In [4]:
global_data_dir = Path("/mnt/ceph/storage/data-in-progress/data-research/web-search/web-archive-query-log/")
global_data_dir

PosixPath('/mnt/ceph/storage/data-in-progress/data-research/web-search/web-archive-query-log')

In [5]:
data_dir = global_data_dir / "focused"
data_dir

PosixPath('/mnt/ceph/storage/data-in-progress/data-research/web-search/web-archive-query-log/focused')

In [6]:
def paths_jsonl(service: str, base_type: str):
    base_path = data_dir / base_type / service
    if not base_path.exists():
        return []
    return [
        path
        for path in base_path.glob("*/*.jsonl.gz")
    ]

In [7]:
def paths_warc(service: str, base_type: str):
    base_path = data_dir / base_type / service
    if not base_path.exists():
        return []
    return [
        path
        for path in base_path.glob("*/*")
        if path.is_dir()
    ]

In [8]:
def count_jsonl(path: Path) -> int:
    with GzipFile(path, "r") as file:
        return sum(1 for _ in file)

In [9]:
def read_jsonl(path: Path) -> int:
    with GzipFile(path, "r") as gzip_file:
        for line in gzip_file:
            url = loads(line)
            yield url

In [10]:
def count_warc(path: Path) -> int:
    lock_path = path / ".lock"
    with lock_path.open("rt") as file:
        return sum(1 for _ in file)

In [11]:
def urls(service: str) -> int:
    paths = paths_jsonl(service, "archived-urls")
    if len(paths) == 0:
        return 0
    return sc.parallelize(paths).map(count_jsonl).sum()

In [12]:
def query_urls(service: str) -> int:
    paths = paths_jsonl(service, "archived-query-urls")
    if len(paths) == 0:
        return 0
    return sc.parallelize(paths).map(count_jsonl).sum()

In [13]:
def unique_query_urls(service: str) -> int:
    paths = paths_jsonl(service, "archived-query-urls")
    if len(paths) == 0:
        return 0
    return sc.parallelize(paths).flatMap(read_jsonl).map(lambda url: url["query"]).distinct().count()

In [14]:
def raw_serps(service: str) -> int:
    paths = paths_warc(service, "archived-raw-serps")
    if len(paths) == 0:
        return 0
    return sc.parallelize(paths).map(count_warc).sum()

In [15]:
def parsed_serps(service: str) -> int:
    paths = paths_jsonl(service, "archived-parsed-serps")
    if len(paths) == 0:
        return 0
    return sc.parallelize(paths).map(count_jsonl).sum()

In [16]:
def search_result_snippets(service: str) -> int:
    paths = paths_jsonl(service, "archived-parsed-serps")
    if len(paths) == 0:
        return 0
    return sc.parallelize(paths).flatMap(read_jsonl).map(lambda serp: len(serp["results"])).sum()

In [17]:
def raw_search_results(service: str) -> int:
    paths = paths_warc(service, "archived-raw-search-results")
    if len(paths) == 0:
        return 0
    return sc.parallelize(paths).map(count_warc).sum()

In [18]:
def parsed_search_results(service: str) -> int:
    paths = paths_jsonl(service, "archived-parsed-search-results")
    if len(paths) == 0:
        return 0
    return sc.parallelize(paths).map(count_jsonl).sum()

In [22]:
def stats(service: str) -> str:
    line = ",".join([
        service, 
        str(urls(service)), 
        str(query_urls(service)), 
        str(unique_query_urls(service)), 
        str(raw_serps(service)),
        str(parsed_serps(service)),
        str(search_result_snippets(service)),
        str(raw_search_results(service)),
        str(parsed_search_results(service)),
    ])
    return line

In [23]:
with Path("numbers.csv").open("wt") as file:
    selected_services = alexa_services
    pool = ThreadPoolExecutor()
    stats_lines = pool.map(stats, selected_services)
    stats_lines = tqdm(
        stats_lines,
        total=len(selected_services),
        desc="Computing stats",
        unit="service",
    )
    stats_lines = list(stats_lines)
    for line in stats_lines:
        file.write(f"{line}\n")

Computing stats:   0%|          | 0/1086 [00:00<?, ?service/s]

In [21]:
global_unique_query_urls = sc.parallelize(alexa_services)\
    .flatMap(lambda service: paths_jsonl(service, "archived-query-urls"))\
    .repartition(1000)\
    .flatMap(read_jsonl).map(lambda url: url["query"])\
    .distinct().count()
print(global_unique_query_urls)

63374140
